In [1]:
from functools import partial
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()
(X_train, y_train) , (X_valid, y_valid)   = (X_train_full[:-5000], y_train_full[:-5000]), (X_train_full[-5000:], y_train_full[-5000:])

X_mean = X_train.mean(axis = 0, keepdims = True)
X_std  = X_train.std(axis = 0, keepdims = True) + 1e-7

X_train = (X_train - X_mean)/(X_std)
X_valid = (X_valid - X_mean)/(X_std)
X_test = (X_test - X_mean)/(X_std)


X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]



In [3]:
CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [4]:
X_train.shape, X_valid.shape

((45000, 32, 32, 3, 1), (5000, 32, 32, 3, 1))

In [5]:
from datetime import datetime

date_time = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")

Lenet

In [6]:
X_train.shape, X_valid.shape

((45000, 32, 32, 3, 1), (5000, 32, 32, 3, 1))

In [15]:
DefaultAlexConv2D = partial(keras.layers.Conv2D, strides=(4, 4) , activation = tf.nn.relu)
DefaultAlexMaxPool2D = partial(keras.layers.MaxPool2D,  strides=(2, 2), padding = 'VALID')
DefaultBatchNormalization = partial(tf.keras.layers.BatchNormalization)
DefaultDropout = partial(tf.keras.layers.Dropout, rate = 0.5)

In [16]:
alex_net_model = keras.models.Sequential([
    DefaultAlexConv2D(filters = 96, kernel_size = (11, 11), input_shape = X_train.shape[1:], padding = 'VALID'),
    DefaultBatchNormalization(),
    DefaultAlexMaxPool2D(pool_size=(3, 3)), 
    DefaultAlexConv2D(filters = 256, kernel_size = (5, 5), padding = 'SAME'),
    DefaultBatchNormalization(),
    DefaultAlexMaxPool2D(pool_size=(3, 3)), 
    DefaultAlexConv2D(filters = 384, kernel_size = (3, 3),  padding = 'SAME'),
    DefaultBatchNormalization(),
    DefaultAlexConv2D(filters = 384, kernel_size = (3, 3),  padding = 'SAME'),
    DefaultBatchNormalization(),
    DefaultAlexConv2D(filters = 256, kernel_size = (3, 3),  padding = 'SAME'),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation=tf.nn.relu),
    DefaultDropout(),
    keras.layers.Dense(4096, activation=tf.nn.relu),
    DefaultDropout(),
    keras.layers.Dense(1000, activation=tf.nn.softmax)
    ], name="alex_net_model_" + date_time)

ValueError: Negative dimension size caused by subtracting 11 from 3 for '{{node conv2d_20/Conv2D/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 4, 4, 1], use_cudnn_on_gpu=true](conv2d_20/Conv2D/Reshape, conv2d_20/Conv2D/Conv2D/ReadVariableOp)' with input shapes: [?,32,3,1], [11,11,1,96].

In [ ]:
alex_net_model.summary()

In [ ]:
alex_net_model.compile(loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'], optimizer=tf.optimizers.Adam())
alex_net_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, workers=10)
score = alex_net_model.evaluate(X_test, y_test)
